In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
train_dataset = torchvision.datasets.MNIST(root='./data/',
                                           train=True,
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='./data/',
                                          train=False,
                                          transform=transforms.ToTensor())

In [5]:
train_lodaer = torch.utils.data.DataLoader(train_dataset, batch_size=100, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False)

In [6]:
# DNN implementation before CNN

class FashionDNN(nn.Module):
  def __init__(self):
    super(FashionDNN, self).__init__()
    self.fc1 = nn.Linear(28*28, 256)
    self.drop = nn.Dropout(0.25)
    self.fc2 = nn.Linear(256, 128)
    self.fc3 = nn.Linear(128, 10)
  def forward(self, input_data):
    out = input_data.view(-1, 28*28)
    out = F.relu(self.fc1(out))
    out = self.drop(out)
    out= F.relu(self.fc2(out))
    out = self.fc3(out)
    return out

In [7]:
learning_rate = 0.001
model = FashionDNN()
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
print(model)

FashionDNN(
  (fc1): Linear(in_features=784, out_features=256, bias=True)
  (drop): Dropout(p=0.25, inplace=False)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=10, bias=True)
)


In [9]:
num_epochs = 5
count = 0
loss_list = list()
iteration_list = list()
accuracy_list = list()

predictions_list = list()
labels_list = list()

for epoch in range(num_epochs):
  for images, labels in train_lodaer:
    images, labels = images.to(device), labels.to(device)

    train = Variable(images.view(100, 1, 28, 28))
    labels = Variable(labels)


    outputs = model(train)
    loss = criterion(outputs, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    count += 1

    if count % 50 == 0:
      total = 0
      correct = 0
      for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        labels_list.append(labels)
        test = Variable(images.view(100, 1, 28, 28))
        outputs = model(test)
        predictions = torch.max(outputs, 1)[1].to(device)
        predictions_list.append(predictions)
        correct += (predictions == labels).sum()
        total += len(labels)
      accuracy = correct * 100 / total
      loss_list.append(loss.data)
      iteration_list.append(count)
      accuracy_list.append(accuracy)

    if count % 500 == 0:
      print("Iteration: {}, Loss: {}, Accuracy: {}%".format(count, loss.data, accuracy))



Iteration: 500, Loss: 0.07734523713588715, Accuracy: 96.5199966430664%
Iteration: 1000, Loss: 0.09772968292236328, Accuracy: 96.98999786376953%
Iteration: 1500, Loss: 0.034866463392972946, Accuracy: 96.89999389648438%
Iteration: 2000, Loss: 0.12592609226703644, Accuracy: 97.39999389648438%
Iteration: 2500, Loss: 0.05790834128856659, Accuracy: 97.12999725341797%
Iteration: 3000, Loss: 0.0577617846429348, Accuracy: 97.22000122070312%


In [10]:
# implement CNN


class FashionCNN(nn.Module):
  def __init__(self):
    super(FashionCNN, self).__init__()
    self.layer1 = nn.Sequential(
        nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.layer2 = nn.Sequential(
        nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )
    self.fc1 = nn.Linear(in_features=64*6*6, out_features=600)
    self.drop = nn.Dropout2d(0.25)
    self.fc2 = nn.Linear(in_features=600, out_features=120)
    self.fc3 = nn.Linear(in_features=120, out_features=10)
  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = out.view(out.size(0), -1)
    out = self.fc1(out)
    out = self.drop(out)
    out = self.fc2(out)
    out = self.fc3(out)
    return out

In [11]:
learning_rate = 0.001
model = FashionDNN()
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

print(model)

FashionDNN(
  (fc1): Linear(in_features=784, out_features=256, bias=True)
  (drop): Dropout(p=0.25, inplace=False)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=10, bias=True)
)


In [12]:
num_epochs = 5
count = 0
loss_list = list()
iteration_list = list()
accuracy_list = list()

predictions_list = list()
labels_list = list()

for epoch in range(num_epochs):
  for images, labels in train_lodaer:
    images, labels = images.to(device), labels.to(device)

    train = Variable(images.view(100, 1, 28, 28))
    labels = Variable(labels)


    outputs = model(train)
    loss = criterion(outputs, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    count += 1

    if count % 50 == 0:
      total = 0
      correct = 0
      for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        labels_list.append(labels)
        test = Variable(images.view(100, 1, 28, 28))
        outputs = model(test)
        predictions = torch.max(outputs, 1)[1].to(device)
        predictions_list.append(predictions)
        correct += (predictions == labels).sum()
        total += len(labels)
      accuracy = correct * 100 / total
      loss_list.append(loss.data)
      iteration_list.append(count)
      accuracy_list.append(accuracy)

    if count % 500 == 0:
      print("Iteration: {}, Loss: {}, Accuracy: {}%".format(count, loss.data, accuracy))



Iteration: 500, Loss: 0.1657533198595047, Accuracy: 94.31999969482422%
Iteration: 1000, Loss: 0.11982455104589462, Accuracy: 95.93999481201172%
Iteration: 1500, Loss: 0.16294515132904053, Accuracy: 96.41999816894531%
Iteration: 2000, Loss: 0.21932020783424377, Accuracy: 96.63999938964844%
Iteration: 2500, Loss: 0.023799702525138855, Accuracy: 97.29000091552734%
Iteration: 3000, Loss: 0.06126715615391731, Accuracy: 97.12999725341797%
